# re
## bad

In [1]:
import re


address = "One Infinite Loop, Cupertino 95014"
city_zip_code_regex = r"^[^,\\]+[,\\\s]+(.+?)\s*(\d{5})?$"

matches = re.match(city_zip_code_regex, address)
if matches:
    print(f"{matches[1]}: {matches[2]}")

Cupertino: 95014


## good

In [2]:
import re

# (?P<city>.+?) set_name

address = "One Infinite Loop, Cupertino 95014"
city_zip_code_regex = r"^[^,\\]+[,\\\s]+(?P<city>.+?)\s*(?P<zip_code>\d{5})?$"

matches = re.match(city_zip_code_regex, address)
if matches:
    print(f"{matches['city']}, {matches['zip_code']}")

Cupertino, 95014


# Đừng thêm ngữ cảnh không cần thiết
## bad

In [3]:
class Car:
    car_make: str
    car_model: str
    car_color: str

## good

In [4]:
class Car:
    make: str
    model: str
    color: str

# Sử dụng các đối số mặc định thay vì dấu tròn hoặc điều kiện
## bad

In [5]:
import hashlib


def create_micro_brewery(name):
    name = "Hipster Brew Co." if name is None else name
    slug = hashlib.sha1(name.encode()).hexdigest()
    # etc.

## good

In [6]:
import hashlib


def create_micro_brewery(name: str = "Hipster Brew Co."):
    slug = hashlib.sha1(name.encode()).hexdigest()
    # etc.

# function(toi da 2 arg, it hon cang tot)

In [7]:
from typing import TypedDict, Text

# python3.8
class MenuConfig(TypedDict):
    
    """
    typing.TypeDict
    
    dung __getitem__ ['attr']
    NO __getattr__ .atrr
    
    A configuration for the Menu.

    Attributes:
        title: The title of the Menu.
        body: The body of the Menu.
        button_text: The text for the button label.
        cancellable: Can it be cancelled?
    """
    title: Text
    body: Text
    button_text: Text
    cancellable: bool


def create_menu(config: MenuConfig):
    title = config["title"]
    body = config["body"]
    button_text = config["button_text"]
    cancellable = config["cancellable"]
    return config
    


create_menu(
    # You need to supply all the parameters
    MenuConfig(
        title="My delicious menu", 
        body="A description of the various items on the menu",
        button_text="Order now!",
        cancellable=True
    )
)

{'title': 'My delicious menu',
 'body': 'A description of the various items on the menu',
 'button_text': 'Order now!',
 'cancellable': True}

# function nen lam chi 1 viec
## bad

In [8]:
from typing import List


class Client:
    active: bool


def email(client: Client) -> None:
    pass


def email_clients(clients: List[Client]) -> None:
    """Filter active clients and send them an email.
    """
    for client in clients:
        if client.active:
            email(client)

## good

In [9]:
from typing import Generator, Iterator


class Client:
    active: bool


def email(client: Client):
    pass


def active_clients(clients: Iterator[Client]) -> Generator[Client, None, None]:
    """Only active clients"""
    return (client for client in clients if client.active)


def email_client(clients: Iterator[Client]) -> None:
    """Send an email to a given list of clients.
    """
    for client in active_clients(clients):
        email(client)

# ten ham phai noi len nhung gi chung lam
## bad

In [10]:
class Email:
    def handle(self) -> None:
        pass

message = Email()
# What is this supposed to do again?
message.handle()

## good

In [11]:
class Email:
    def send(self) -> None:
        """Send this message"""

message = Email()
message.send()

# Các hàm chỉ nên là một cấp độ trừu tượng
## bad

In [12]:
def parse_better_js_alternative(code: str) -> None:
    regexes = [
        # ...
    ]

    statements = code.split('\n')
    tokens = []
    for regex in regexes:
        for statement in statements:
            pass

    ast = []
    for token in tokens:
        pass

    for node in ast:
        pass

## good

In [13]:
from typing import Tuple, List, Text, Dict


REGEXES: Tuple = (
   # ...
)


def parse_better_js_alternative(code: Text) -> None:
    tokens: List = tokenize(code)
    syntax_tree: List = parse(tokens)

    for node in syntax_tree:
        pass


def tokenize(code: Text) -> List:
    statements = code.split()
    tokens: List[Dict] = []
    for regex in REGEXES:
        for statement in statements:
            pass

    return tokens


def parse(tokens: List) -> List:
    syntax_tree: List[Dict] = []
    for token in tokens:
        pass

    return syntax_tree

# Không sử dụng flags làm tham số hàm
## bad

In [14]:
from typing import Text
from tempfile import gettempdir
from pathlib import Path

# no argument temp
def create_file(name: Text, temp: bool) -> None:
    if temp:
        (Path(gettempdir()) / name).touch()
    else:
        Path(name).touch()

## good

In [15]:
from typing import Text
from tempfile import gettempdir
from pathlib import Path


def create_file(name: Text) -> None:
    Path(name).touch()


def create_temp_file(name: Text) -> None:
    (Path(gettempdir()) / name).touch()

# Avoid side effects - tac dung phu
## bad

In [16]:
fullname = "Ryan McDermott"

def split_into_first_and_last_name() -> None:
    global fullname #effect
    
    fullname = fullname.split()

split_into_first_and_last_name()

print(f"{fullname = }")  # ["Ryan", "McDermott"]

fullname = ['Ryan', 'McDermott']


## good

In [17]:
from typing import List, AnyStr


def split_into_first_and_last_name(name: AnyStr) -> List[AnyStr]:
    return name.split()

fullname = "Ryan McDermott"
name, surname = split_into_first_and_last_name(fullname)

print(f"{name = } - {surname = }")  # => Ryan McDermott
print(f"{fullname = }")

name = 'Ryan' - surname = 'McDermott'
fullname = 'Ryan McDermott'


## also good

In [18]:
from dataclasses import dataclass

@dataclass
class Person:
    name: str

    @property
    def name_as_first_and_last(self) -> list:
        return self.name.split() 


# The reason why we create instances of classes is to manage state!
person = Person("Dung Thong")
print(f"{person.name = }")  # => "Dung Thong"
print(f"{person.name_as_first_and_last = }")

person.name = 'Dung Thong'
person.name_as_first_and_last = ['Dung', 'Thong']


# SOLID

## 1. Single Responsibility Principle(SRP)

In [19]:
note = """
    Mỗi lớp chỉ nên chịu trách nhiệm về một nhiệm vụ cụ thể nào đó mà thôi.
"""

class AccountDB:
   """Account DB management class """

   def get_account_number(self, _id):
       """Get account number"""
       pass

   def account_save(self, obj):
       """Save account number into DB"""
       pass


class Account:
   """Demo bank account class """

   def __init__(self, account_no: str):
       self.account_no = account_no
       self._db = AccountDB()

   def get_account_number(self):
       """Get account number"""
       return self.account_no

   def get(self, _id):
       
       return self._db.get_account_number(_id=_id)

   def save(self):
       """account save"""
       self._db.account_save(obj=self)

## 2. Open and Closed Principle(OCP)

In [20]:
note = """
    Chúng ta nên hạn chế việc chỉnh sửa bên trong một Class hoặc Module có sẵn, 
    thay vào đó hãy xem xét mở rộng chúng.
    
    Khi cần thêm tính năng mới, ta nên kế thừa và mở rộng các module/class có sẵn thành các module con lớn hơn.
    Các module/class con vừa có các đặc tính của lớp cha (đã được kiểm chứng đúng đắn),
    vừa được bổ sung tính năng mới phù hợp với yêu cầu.
    
    
    def give_discount(self):
    
       if self.customer == 'normal':
           return self.price * 0.2
       elif self.customer == 'vip':
           return self.price * 0.4
       elif self.customer ==  'supvip':
           return self.price * 0.8

"""

class Discount:
   """Demo customer discount class"""

   def __init__(self, customer, price):
       self.customer = customer
       self.price = price
        
   def get_discount(self):
       """A discount method"""
       return self.price * 0.2


class VIPDiscount(Discount):
   """Demo VIP customer discount class"""

   def get_discount(self):
       """A discount method"""
       return super().get_discount() * 2
    
    
class SuperVIPDiscount(VIPDiscount):
   """Demo super vip customer discount class"""

   def get_discount(self):
       """A discount method"""
       return super().get_discount() * 4

## 3. Liskov Substitution Principle(LSP)

In [21]:
note = """
     Các instance của lớp con có thể thay thế được instance lớp cha 
     mà vẫn đảm bảo tính đúng đắn của chương trình.
     thuc ra la tinh da hinh
     
"""

class Vehicle:
   """
   xay dung cac methods dung chung, cho cac childs class
   con cac childs class thi xay dung cac methods cho rieng no    
        
   """

   def __init__(self, name: str, speed: float):
       self.name = name
       self.speed = speed
        
   def get_name(self) -> str:
       """Get vehicle name"""
       return f"The vehicle name {self.name}"

   def get_speed(self) -> str:
       """Get vehicle speed"""
       return f"The vehicle speed {self.speed}"
    
    
class VehicleWithoutEngine(Vehicle):
   """A demo Vehicle without engine class"""

   def start_moving(self):
      """Moving"""
      raise NotImplemented
        
        
class VehicleWithEngine(Vehicle):
   """A demo Vehicle engine class"""

   def engine(self):
      """A vehicle engine"""
      pass
    
   def start_engine(self):
      """A vehicle engine start"""
      self.engine()
    
    
class Car(VehicleWithEngine):
   """A demo Car Vehicle class"""

   def start_engine(self):
       pass


class Bicycle(VehicleWithoutEngine):
   """A demo Bicycle Vehicle class""" 

   def start_moving(self):
       pass

## 4. Interface Segregation Principle(ISP)

In [22]:
note = """
    Thay vì dùng 1 interface lớn, ta nên tách thành nhiều interface nhỏ, với nhiều mục đích cụ thể
"""

class BankAccount:
   """A demo Bank Account class"""

   def __init__(self, balance: float, account: str):
       self.account = {f"{account}": balance}
                       
   def balance(self, account: str):
       """Get current balance"""
                       
       raise NotImplemented
                       
                       
class Deposit(BankAccount):          
   """A demo circle class"""
                       
   def balance(self, account: str):
      """Get current balance"""
                       
      return self.account.get(account)
                       
   def deposit(self, amount: float, account: str):
       """Deposit a new amount"""
                       
       current = self.balance(account)
       new_amount = current + amount
       self.account.update({account: new_amount})

    
class Child(BankAccount):
    pass


child = Child(1.5, 'phong')

## 5. Dependency Inversion Principle(DIP)

In [23]:
note = """
    
"""

class BackendDeveloper:
    """This is a low-level module"""
    
    @staticmethod
    def python():
        print("Writing Python code")
        
        
class FrontendDeveloper:
    """This is a low-level module"""
    
    @staticmethod
    def javascript():
        print("Writing JavaScript code")
        
        
class Project:
    """This is a high-level module"""
    
    def __init__(self):
        self.backend = BackendDeveloper()
        self.frontend = FrontendDeveloper()
        
    def develop(self):
        self.backend.python()
        self.frontend.javascript()
        
        return "Develop codebase"
    
    
project = Project()
print(project.develop())


Writing Python code
Writing JavaScript code
Develop codebase


### 5.1 polymorphism - tinh da hinh

In [24]:
note = """
    thuc ra thi xay dung cac methods voi ten giong nhau...
    ket hop voi decorator, @property...
"""

class BackendDeveloper:
   """This is a low-level module"""

   def develop(self):
       self.__python_code()
    
   @staticmethod
   def __python_code():
       print("Writing Python code")
    
    
class FrontendDeveloper:
   """This is a low-level module"""

   def develop(self):
       self.__javascript()
    
   @staticmethod
   def __javascript():
       print("Writing JavaScript code")
    
    
class Developers:
   """An Abstract module"""

   def __init__(self):
       self.backend = BackendDeveloper()
       self.frontend = FrontendDeveloper()
        
   def develop(self):
       self.backend.develop()
       self.frontend.develop()
    
    
class Project:
   """This is a high-level module"""

   def __init__(self):
       self.__developers = Developers()
    
   def develops(self):
       return self.__developers.develop()
    
    
project = Project()
print(project.develops())

Writing Python code
Writing JavaScript code
None


### 5.2

In [25]:
class NewsPerson:
   """This is a high-level module"""

   @staticmethod
   def publish(news: str, publisher=None) -> None:
       print(publisher.publish(news=news))
        
        
class NewsPaper:
   """This is a low-level module"""

   @staticmethod
   def publish(news: str) -> None:
       print("{} news paper".format(news))
        
        
class Facebook:
   """This is a low-level module"""

   @staticmethod
   def publish(news: str) -> None:
       print(f"{news} - share this post on {news}")
        
        
person = NewsPerson()
person.publish("hello", NewsPaper())
person.publish("facebook", Facebook())

hello news paper
None
facebook - share this post on facebook
None


## Nguyên tắc YAGNI:


In [26]:
note = """
    YAGNI = You Aren’t Gonna Need It
    Nguyên tắc này là dự án của bạn đang làm thì chỉ cần tập trung xây dựng chức năng quyết vấn đề
    ở thời điểm hiện tại
    vấn đề mà khách hàng cần giải quyết, không cần lãng phí thời gian vào một chức năng "Có thể sử dụng đến".
=>  Đừng tự vẽ việc cho mình.
"""

## Nguyên tắc KISS

In [27]:
note = """
    chi nho van de, chia nho, chia nao!!!!
"""

## Nguyên tắc DRY:

In [28]:
note = """
    dua code vao function hoac dung inheritance, chi call lai khi dung
    tranh code lai doan cau do
"""